In [ ]:
!pip install --no-cache-dir -q transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 202.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 110.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'bragovo/qqq'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model.cuda() if torch.cuda.is_available() else None
model.eval();

def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [ ]:
!pip install -q wandb

In [ ]:
text = """Здравствуйте, техническая поддержка СПбГУ, слушаю вас. Здравствуйте, у меня на компьютере интернет не работает, что делать? Вы находитесь в общежитии? Да, в тринашке. На компьютере нет подключения к сети или именно к интернету? А в чем разница? Снизу справа на панели задач у значка интернета красный крестик или желтый треугольник? Треугольник. Хорошо, назовите свой номер абонента. 1707.4.20. Сейчас проверю. Ярослав Гусь? Да, все верно. У вас задолженность по оплате. Точно, сейчас оплачу, спасибо. Все, до свидания."""
text = text.lower()
text

'здравствуйте, техническая поддержка спбгу, слушаю вас. здравствуйте, у меня на компьютере интернет не работает, что делать? вы находитесь в общежитии? да, в тринашке. на компьютере нет подключения к сети или именно к интернету? а в чем разница? снизу справа на панели задач у значка интернета красный крестик или желтый треугольник? треугольник. хорошо, назовите свой номер абонента. 1707.4.20. сейчас проверю. ярослав гусь? да, все верно. у вас задолженность по оплате. точно, сейчас оплачу, спасибо. все, до свидания.'

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
our_dataset = load_dataset('json', data_files='summary_data.json')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-950c5b65da2d19c8/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
our_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summa'],
        num_rows: 4
    })
})

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summa"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_our_dataset = our_dataset.map(preprocess_function, batched=True)

In [ ]:
tokenized_our_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summa', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=MODEL_NAME)

In [ ]:
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.2 MB/s eta 0:00:00


In [ ]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_model",
    evaluation_strategy="steps",
    eval_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=400,
    predict_with_generate=True,
    push_to_hub=False,
    logging_steps=100,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_our_dataset["train"],
    eval_dataset=tokenized_our_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
100,0.843900,0.014525,0.000000,0.000000,0.000000,0.000000,17.500000
200,0.036500,0.000448,0.000000,0.000000,0.000000,0.000000,17.500000
300,0.014700,0.000199,0.000000,0.000000,0.000000,0.000000,17.500000
400,0.009200,0.000173,0.000000,0.000000,0.000000,0.000000,17.500000


TrainOutput(global_step=400, training_loss=0.2260865457355976, metrics={'train_runtime': 121.1431, 'train_samples_per_second': 13.208, 'train_steps_per_second': 3.302, 'total_flos': 412090417152000.0, 'train_loss': 0.2260865457355976, 'epoch': 400.0})

In [ ]:
%%time
text = """Добрый день, техническая поддержка СПбГУ, слушаю вас.
Добрый день. Я забыла, к сожалению, свой пароль от почты, что мне нужно сделать?
Скажите пожалуйста свои фамилию, имя, отчество?
Филатова Мария Евгеньевна.
Хорошо, какой у вас факультет?
Матмех.
Какой у вас номер группы?
20 б 07 мм.
Ага, хорошо, с завтрашнего дня можете зайти в деканат, показать студенческий билет, и вам выдадут новый пароль.
А дистанционно никак не получится это сделать?
Простите, но мы не можем без удостоверения личности выдать вам новый пароль.
Ладно, хорошо, спасибо, до свидания.
До свидания."""
print(summarize(text))

Филатова Мария Евгеньевна забыла свой пароль от почты.
CPU times: user 941 ms, sys: 0 ns, total: 941 ms
Wall time: 1 s


In [ ]:
%%time
text = """Добрый день, техническая поддержка СПбГУ, слушаю вас.
Здравствуйте, у меня не открывается Блэкборд, можете проверить?
Подождите секунду. Сейчас проверим. Да, есть проблема с работой серверов. Работа будет восстановлена после проведения технических работ.
Буду ждать, до свидания.
До свидания."""
print(summarize(text))

Не открывается Блэкборд.
CPU times: user 270 ms, sys: 0 ns, total: 270 ms
Wall time: 276 ms


In [ ]:
!git clone https://github.com/spbu-smart-assistant/support-chatbot.git

NotImplementedError: ignored